In [6]:
import gurobipy as gp
import numpy as np
import matplotlib.pyplot as plt
import random
from random import randint
from sklearn.datasets import make_blobs
from utility import *

In [7]:
#points = load_points('C:/Users/franc/Desktop/prova.txt')

points = load_points(f'C:/Users/franc/Documents/GitHub/Ricerca_Operativa_2022/Ricerca_Operativa_2022/benchmark/benchmark{1}.txt')

K = 7
N = len(points)

dist = {}

for i in range(N):
    for j in range(N):
        dist[(i,j)] = np.sum((points[i]-points[j])**2)

In [8]:
model = gp.Model()
x = model.addVars(N,N, vtype = gp.GRB.BINARY, name = "X") #i elemento , j centroide
y = model.addVars(N, vtype = gp.GRB.BINARY, name = "Y")

obj = model.setObjective(x.prod(dist), gp.GRB.MINIMIZE)

model.addConstrs((sum(x[i,j] for j in range(N)) == 1 for i in range(N)))
model.addConstr(sum(y[j] for j in range(N))==K)
model.addConstrs((x[i,j]<=y[j] for i in range(N) for j in range(N)))

model.optimize()


Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 250501 rows, 250500 columns and 750500 nonzeros
Model fingerprint: 0x117fef5e
Variable types: 0 continuous, 250500 integer (250500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 3e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+00]
Presolve time: 2.30s
Presolved: 250501 rows, 250500 columns, 750500 nonzeros
Variable types: 0 continuous, 250500 integer (250500 binary)
Found heuristic solution: objective 407989.49104

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   39460    3.1541145e+05   0.000000e+00   4.777412e+08      5s
Concurrent spin time: 0.01s

Solved with dual simplex

Root relaxation: objective 2.321598e+04, 16045 iterations, 5.51 s

In [9]:
#Converto la soluzione nel formato utilizzato dalle euristiche:

centroids = []
for j in range(N):
    if y[j].X == 1:
        centroids.append(j)

sol = np.zeros(N)

for i in range(N):
    for cluster,j in enumerate(centroids):
        if x[i,j].X == 1:
            sol[i] = cluster


print("{:.5E}".format(squared_inner_distance(sol, points, K)))

1.54028E+04
